## Solving Poisson for 4 dimensional P1 Pentatopes
In this notebook we are going to solve the simple Laplace equation  
$$ 
-\Delta u = f
 $$  
with Dirichlet boundary data $u_D$ on $\partial\Omega$.


First we create a 4d semistructured mesh we use an unstructured mesh discretization of $\Omega$ and expand it to a 4d mesh

In [6]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import numpy as np
from netgen.csg import unit_cube
from ngsolve import Mesh,VOL,specialcf

ngmesh = Mesh(unit_cube.GenerateMesh(maxh=0.25))
m=4
mesh = UnstructuredHypertriangleMesh(m,ngmesh)

Now we show some properties the mesh provides

In [7]:
print("Number of points in the mesh " , len(mesh.points))
print("Number of Vertices ", len(mesh.vertices))
print("Number of elements ", len(mesh.hypercells))
print("Getting all the  boundary dofs:", mesh.bndry_vertices)
print("getting only the initial DOFS ", mesh.initial_bndry_vertices)

Number of points in the mesh  705
Number of Vertices  705
Number of elements  7280
Getting all the  boundary dofs: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 

Let us initialize a Finite Element Space. This time we choose the P1-Hypertriangle Space. For P1 elements every vertex becomes a degree of freedom and our reference element is the P1 Pentatope

<img src="../graphics/Pentatope.png" width="600">


In [8]:
from methodsnm.fes import *
fes = P1_Hypertriangle_Space(mesh)
print(fes.ndof)
print(fes.fe)

705
P1 4D-Simplex Finite Element


We are constructing an easy example to verify our implementation 
We consider the analytical solution
$$
u(x,y,z,t) = \cos(\pi x)\,\cos(\pi y)\,\cos(\pi z)\,\cos(\pi t),
$$
which satisfies the equation
$$
-\Delta u(x,y,z,t) = 4\pi^2 \cos(\pi x)\,\cos(\pi y)\,\cos(\pi z)\,\cos(\pi t).
$$


In [9]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin

blf = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf += LaplaceIntegral(c)
blf.assemble()

lf = LinearForm(fes)
f = GlobalFunction(lambda x: 4*pi**2*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3]), mesh = mesh)
lf += SourceIntegral(f)
lf.assemble()

Some helper function we are going to use to define the freedofs

In [10]:
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

Now we are going to set the boundary conditions in our case we take $u_D = u(x,t)$. And we decompose our solution $u =u_D +u_0$
So we solve instead of $Au = F \Leftrightarrow  Au_0 =F- Au_D$ Since $u_0$ only lives on the inner $\Omega$ we will only solve for the freedofs living in $\Omega$ (freedofs)

In [11]:

uh = FEFunction(fes)
freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
uh._set(lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3]), True)
from scipy.sparse.linalg import spsolve
from methodsnm.solver import solve_on_freedofs
res = lf.vector - blf.matrix.dot(uh.vector)
uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)


solving the linear system gives us our approximated u. We can now compute the L2 error ...

In [12]:
from methodsnm.forms import compute_difference_L2

uex =  GlobalFunction(lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3]), mesh = mesh)
l2diff = compute_difference_L2(uh, uex, mesh, intorder = 3)

print("l2diff =", l2diff)

l2diff = 0.049616899811011604


And we see a pretty descent approximation. We now can obviously test the convergencerate by refining the mesh or try to solve another PDE as for example in the [convection-diffusion](/demos/convection-diffusion/con-dominated-problem4d.ipynb) example.

### Convergence rate

Let us verify that our method reproduces the expected analytical convergence rate.  
For the Poisson problem with Dirichlet boundary conditions and a standard continuous Galerkin discretization with linear finite elements  
$u_h \in P_1 \ \text{(globally continuous)},$  
the classical a priori error estimate states

$$
\|u - u_h\|_{H^1(\Omega)} \le C h\, \|u\|_{H^2(\Omega)},
$$

and, by the Aubin–Nitsche duality argument, the $L^2$ error satisfies

$$
\|u - u_h\|_{L^2(\Omega)} \le C h^2\, \|u\|_{H^2(\Omega)}.
$$

Thus, when the mesh size $h$ is refined by a factor of $2$, we expect the $L^2$ error to decrease by a factor of

$$
2^2 = 4.
$$

Our numerical experiments should therefore exhibit a quadratic convergence rate in the $L^2$ norm.


In [15]:
from ngsolve import Mesh,VOL,specialcf
h = 0.5
m = 2
for i in range(3):
    ngmesh = Mesh(unit_cube.GenerateMesh(maxh=h))
    mesh = UnstructuredHypertriangleMesh(m,ngmesh)
    fes = P1_Hypertriangle_Space(mesh=mesh)

    blf = BilinearForm(fes)
    c = GlobalFunction(lambda x: 1, mesh = mesh)
    blf += LaplaceIntegral(c)
    blf.assemble()

    lf = LinearForm(fes)
    f = GlobalFunction(lambda x: 4*pi**2*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3]), mesh = mesh)
    lf += SourceIntegral(f)
    lf.assemble()

    uh = FEFunction(fes)
    freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
    uh._set(lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3]), True)
    from scipy.sparse.linalg import spsolve
    from methodsnm.solver import solve_on_freedofs
    res = lf.vector - blf.matrix.dot(uh.vector)
    uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)
    l2diff = compute_difference_L2(uh, uex, mesh, intorder = 3)
    print("l2diff =", l2diff)
    m = 2*m
    h = h/2


l2diff = 0.2089885323411974
l2diff = 0.049616899811011604
l2diff = 0.01314047374246834


Now we see that our numerical results match our expectations